# Homework 5

Please follow the guidelines in Sec ***9. Homework Projects*** of the Syllabus. 

Homework projects must be submitted via Canvas as a *working* Jupyter notebook.

## Question 1

In this question, we generalize Newton's method to two dimensions. 

As in the 1D case discussed in class, we begin with the Taylor expansion for a function of two variables
$$
    f(x_{n+1}, y_{n+1}) = f(x_n, y_n) + \frac{\partial f}{\partial x}(x_n, y_n)\cdot (x_{n+1} - x_n)  
        + \frac{\partial f}{\partial y}(x_n, y_n) \cdot (y_{n+1} - y_n) + \cdots
$$
Assuming $f(x_{n+1}, y_{n+1}) \approx 0$, which means that the next interation $(x_{n+1}, y_{n+1})$ gives us a better approximation for a zero of the function $f$ than the previous iteration $(x_{n}, y_{n})$. Hence, we get the equation for $(x_{n+1}, y_{n+1})$:
$$
 0 = f(x_n, y_n) + \frac{\partial f}{\partial x}(x_n, y_n)\cdot (x_{n+1} - x_n)  
        + \frac{\partial f}{\partial y}(x_n, y_n) \cdot (y_{n+1} - y_n),
$$
which can be re-written in the matrix form
$$
\begin{pmatrix}
    \frac{\partial f}{\partial x}(x_n, y_n) &
    \frac{\partial f}{\partial y}(x_n, y_n)
\end{pmatrix}
\begin{pmatrix}
    x_{n+1} - x_n \\
    y_{n+1} - y_n
\end{pmatrix}
= -f(x_n, y_n).
$$


In this case, Newton’s method can be implemented as follows: Given $(x_n, y_n)$, solve 
$$
    \begin{pmatrix}
    \frac{\partial f}{\partial x}(x_n, y_n) &
    \frac{\partial f}{\partial y}(x_n, y_n)
\end{pmatrix}
\vec{\Delta}
= -f(x_n, y_n)
$$
for $\vec{\Delta}$. (*Hint:* use the command `\`.) Then, update 
$$
    x_{n+1} = x_{n} + \Delta_1, \qquad y_{n+1} = y_{n} + \Delta_2.
$$

Code this method up and use it to numerically find the zero of
$$
    f(x, y) = x^2 + y^2.
$$

In [57]:
using LinearAlgebra
function newton_min(∇f::Function, Hessian::Function, 𝐱₀::Vector; max_nsteps::Int = 1000, abstol::Real = 1e-10)
     
    𝐱ₙ = 𝐱ₙ₊₁ = 𝐱₀
    
    history_𝐱 = [𝐱₀]
    
    for step = 1:max_nsteps
        
        𝐱ₙ₊₁ = 𝐱ₙ - Hessian(𝐱ₙ[1], 𝐱ₙ[2]) \ ∇f(𝐱ₙ[1], 𝐱ₙ[2])
        
        push!(history_𝐱, 𝐱ₙ₊₁)
        
        if norm(𝐱ₙ₊₁ - 𝐱ₙ) < abstol
           return hcat(history_𝐱...) 
        end
        
        𝐱ₙ = 𝐱ₙ₊₁ 
    end
    
    println("Maximum number of iterations is reached")
    
    return hcat(history_𝐱...)
end

∇f(x,y) = [2*x; 2*y]
Hessian(x,y) = [2 0; 0 2]
x₀ = [1.0, 1.0]

k = newton_min(∇f, Hessian, x₀)
solution = k[1,end] , k[2,end]

(0.0, 0.0)

## Question 2: extra credit for 3170, required for 6170

As discussed in class, Newton’s method also allows to find the complex roots. We just need to pick some initial (complex) guess and perform the iterations
$$
	z_{n+1} = z_n - \frac{f(z_n)}{f'(z_n)}.
$$ 

This iteration scheme hides a beautiful fractal structure, known as [the Newton fractal](https://en.wikipedia.org/wiki/Newton_fractal). Let’s uncover it in the case of function

$$
	f(z) = z^3 - 1.
$$

The polynomial $f(z)$ has three complex roots ($-1/2 \pm i\sqrt{3}/2$ and $1$). For every complex initial guess, Newton’s iteration sequence $z_n$ converges to one of those roots. The Newton fractal appears if we label (i.e., color) every point $c$ in the complex plane by the root to which Newton's method converges when started from $c$.

By adapting the in-class code drawing the Mandelbrot set, visualize the Newton fractal for $ f(z) = z^3 - 1$ in the following way: Use `x[k] + im * y[j]` as the starting point for Newton's method and the imaginary (`imag`) part of $z_{200}$ to color the pixel `image[j, k]`. It is recommend to use `x = range(-5, 5, 1000)` and `y = range(-5, 5, 1000)`.

Hint: The resulting image should look like [this](https://en.wikipedia.org/wiki/Newton_fractal#/media/File:Newtroot_1_0_0_m1.png).

In [58]:
x = range(-5, 5, 1000)
y = range(-5, 5, 1000)

image = zeros(length(y), length(x))

Threads.@threads for k = 1:length(x)
    for j = 1:length(y)
    
        z0 = x[k] + im * y[j]
        

        # Iterating
        zₙ = 0im
        for step = 1:200
            f(z) = (z^3) - 1
            f_1(z) = 3 * z^2
            dzₙ = f(zₙ)/f_1(zₙ) 
        end
        
        # Coloring
        val = abs(zₙ - dzₙ)
        if val < 10
            image[j, k] = val
        else
            image[j, k] = 10
        end
    end
end


using PyPlot

imshow(image, origin="lower", cmap="jet")
colorbar()
show()

LoadError: TaskFailedException

[91m    nested task error: [39mUndefVarError: dzₙ not defined
    Stacktrace:
     [1] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m.\[39m[90m[4mIn[58]:21[24m[39m[90m [inlined][39m
     [2] [0m[1m(::var"#231#threadsfor_fun#128"{var"#231#threadsfor_fun#127#129"{UnitRange{Int64}}})[22m[0m[1m([22m[90mtid[39m::[0mInt64; [90monethread[39m::[0mBool[0m[1m)[22m
    [90m   @ [39m[35mMain[39m [90m.\[39m[90m[4mthreadingconstructs.jl:84[24m[39m
     [3] [0m[1m#231#threadsfor_fun[22m
    [90m   @ [39m[90m.\[39m[90m[4mthreadingconstructs.jl:51[24m[39m[90m [inlined][39m
     [4] [0m[1m(::Base.Threads.var"#1#2"{var"#231#threadsfor_fun#128"{var"#231#threadsfor_fun#127#129"{UnitRange{Int64}}}, Int64})[22m[0m[1m([22m[0m[1m)[22m
    [90m   @ [39m[90mBase.Threads[39m [90m.\[39m[90m[4mthreadingconstructs.jl:30[24m[39m

In [ ]:
using PyPlot
maxiter=50
tol=1e-6 
roots=[1.0, -1.0, im, -im]
x = range(-5, 5, 1000)
y = range(-5, 5, 1000)
f(z) = z^3 - 1.0

# define the derivative of the function
df(z) = 3z^2
z = [x + im*y' for y in y, x in x]

# initialize arrays to store the iteration count and the root index for each pixel
count = zeros(Int, size(z))
root_index = zeros(Int, size(z))

    for i in 1:maxiter
        z = z .- f(z) ./ df(z)
        for j in 1:length(roots)
            mask = (count .== 0) .& (abs(z - roots[j]) .< tol)
            count[mask] .= i
            root_index[mask] .= j
        end
    end

    plot(z, count, seriestype=:heatmap, color=:rainbow, c=:blues, legend=:none)

LoadError: MethodError: no method matching -(::Matrix{ComplexF64}, ::Float64)
For element-wise subtraction, use broadcasting with dot syntax: array .- scalar
[0mClosest candidates are:
[0m  -([91m::T[39m, ::T) where T<:Union{Float16, Float32, Float64} at float.jl:384
[0m  -([91m::AbstractGray{Bool}[39m, ::Number) at C:\Users\tkeeg\.julia\packages\ColorVectorSpace\QI5vM\src\ColorVectorSpace.jl:342
[0m  -([91m::AbstractGray[39m, ::Number) at C:\Users\tkeeg\.julia\packages\ColorVectorSpace\QI5vM\src\ColorVectorSpace.jl:340
[0m  ...

## Question 3: extra credit 

Derive and implement Newton's method for solving the system of equations 
$$
\begin{cases}
    f(x) = 0 \\
    g(x) = 0.
\end{cases}
$$

Use the code to numerically solve

$$
\begin{cases}
    f(x) = x^{3}+ 8.5 x^{2}+ 10.0 x- 37.5 = 0 \\
    g(x) = x^{3}- 6.0 x^{2}+ 8.75 x- 3.0 = 0.
\end{cases}
$$

*Hint:* Follow the logic outlined in Question 1 by strating with the Taylor expansions 
$$
\begin{cases}
    f(x_{n+1}) = f(x_n) + f'(x_n)\cdot (x_{n+1} - x_n) + \cdots \\
    g(x_{n+1}) = g(x_n) + g'(x_n)\cdot (x_{n+1} - x_n) + \cdots.
\end{cases}
$$